In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
path = '/content/drive/My Drive/STAT457'
os.chdir(path)
os.listdir()

['data',
 'trained.ipynb',
 'csvData.csv',
 'tfidf_xgb.csv',
 'new.csv',
 'tfidf.csv',
 'data.ipynb',
 'full_tfidf.ipynb',
 'bert.csv',
 'full_bert.ipynb',
 'Project2']

In [ ]:
!pip install transformers

In [ ]:
# https://towardsdatascience.com/multi-label-multi-class-text-classification-with-bert-transformer-and-keras-c6355eccb63a
# https://huggingface.co/bert-base-uncased

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from sklearn.model_selection import train_test_split
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
full = pd.read_csv("data/W22_P2_full.csv",encoding='gb18030')
description = full[["description"]]

In [ ]:
xy_train_df = pd.read_csv('data/W22_P2_train.csv', encoding='gb18030', engine='python')
xy_train_df.columns=['id','genre','description']
temp = pd.get_dummies(xy_train_df.genre)
xy_train_df = pd.concat([xy_train_df, temp], axis=1)
print('Shape : ',xy_train_df.shape)
xy_train_df.head()

Shape :  (9998, 7)


,id,Genre,Synopsis,comedy,documentary,drama,short
0,9014,documentary,Docu-drama inspired by the life of Bartolomeo...,0,1,0,0
1,2923,drama,Jos鑼?is a passionate young man in the Dominic...,0,0,1,0
2,313,short,"Imposing, austere gray buildings dominate a s...",0,0,0,1
3,8292,short,"In the multiplex era, a few days prior to his...",0,0,0,1
4,4131,documentary,Edmund Purdom narrates a pseudo-documentary a...,0,1,0,0


In [ ]:
def clean(df):
    descriptions = []
    for description in df['description']:       
        #remove special characters
        clean_text = re.sub("[^a-zA-Z]"," ", description)
        descriptions.append(clean_text)
    return descriptions

def replace(df, new):
  df_temp = copy.deepcopy(df)
  df_temp["description"] = new
  return df_temp

In [ ]:
train_clean_sentence = clean(xy_train_df)
xy_train_new = replace(xy_train_df, train_clean_sentence)

In [ ]:
CLASS_NAME = list(set(xy_train_new.genre))
CLASS_NAME

[' short ', ' documentary ', ' drama ', ' comedy ']

In [ ]:
EPOCH = 5
BATCH = 32

In [ ]:
data = xy_train_new.copy()
data = data[['description','genre']]

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data['genre'] = le.fit_transform(data.genre)

In [ ]:
# Split into train and test
train_data, val_data = train_test_split(data, test_size = 0.2, stratify = data['genre'], random_state = 0)

y_val = le.inverse_transform(val_data['genre'])

In [ ]:
model_name = 'bert-base-uncased'
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

In [ ]:
max_length = 200

config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)

bert = transformer_model.layers[0]

In [ ]:
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}

bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)

genre = Dense(units=len(train_data.genre.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='genre')(pooled_output)
outputs = {'genre': genre}
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')
model.summary()

In [ ]:
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

loss = {'genre': CategoricalCrossentropy(from_logits = True)}
metric = {'genre': CategoricalAccuracy('accuracy')}

model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

y_genre = to_categorical(train_data['genre'])

# Tokenize the training data
x_train = tokenizer(
    text=train_data['description'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

history = model.fit(
    x_train={'input_ids': x_train['input_ids']},
    y={'genre': y_genre},
    validation_split=0.2,
    batch_size=BATCH,
    epochs=EPOCH)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "BERT_MultiLabel_MultiClass"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, 200)]             0         
                                                                 
 bert (TFBertMainLayer)      TFBaseModelOutputWithPoo  109482240 
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             200, 768),                          
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                        

In [ ]:
y_genre.shape

(9998, 4)

In [ ]:
x_val = tokenizer(
    text=val_data['description'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

# Run evaluation
model.predict(x_train={'input_ids': x_val['input_ids']})
y_predicted_raw = model.predict(x_train={'input_ids': x_val['input_ids']})

y_predicted = le.inverse_transform([np.argmax(y_predicted_raw['genre'][index,:], axis=None, out=None) for index in np.arange(len(y_predicted_raw['genre']))])
y_predicted

NameError: ignored

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_predicted, target_names=CLASS_NAME))

NameError: ignored

In [ ]:
y_predicted.shape

(2000,)

In [ ]:
test_raw_df = pd.read_csv('data/W22_P2_test.csv', encoding='gb18030', engine='python')
test_raw_df.columns=['id','description']
print('Shape : ',test_raw_df.shape)
test_raw_df.head()

Shape :  (6000, 2)


,id,Synopsis
0,15692,According to the text of St鑼卲hane E.Roy Nine ...
1,15928,"Dateline # Austin, TX. Barefoot Bobby Briggs,..."
2,11134,"In Campina Grande, Northeastern of Brazil, th..."
3,14626,Frankie Reno was a child star on a TV show. B...
4,11338,Coming from an upper-middle class background ...


In [ ]:
test = tokenizer(
    text=test_raw_df['description'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

model.predict(x={'input_ids': test['input_ids']})
y_predicted_raw = model.predict(x={'input_ids': test['input_ids']})

y_predicted = le.inverse_transform([np.argmax(y_predicted_raw['genre'][index,:], axis=None, out=None) for index in np.arange(len(y_predicted_raw['genre']))])

array([' comedy ', ' documentary ', ' documentary ', ..., ' documentary ',
       ' drama ', ' comedy '], dtype=object)

In [ ]:
y_predicted[0]

' comedy '

In [ ]:
test_id = test_raw_df[["id"]]
test_id = [int(test_id.loc[i]) for i in range(len(test_id))]

In [ ]:
import csv
csvFile = open("bert1.csv", "w")            #创建csv文件
writer = csv.writer(csvFile)                  #创建写的对象
#先写入columns_name
writer.writerow(["id", "genre"])     #写入列的名称
for i in range(len(y_predicted)):
    writer.writerow([test_id[i], y_predicted[i]])
csvFile.close()